In [1]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV ,KFold
import pandas as pd
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.calibration import LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('/Users/shahadaleissa/Downloads/Code/Data/colon-dataset-processed.csv')
le = LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])
df

,Class,Age,p16540_C/C,p16540_G/C,p16540_G/G,p16580_C/C,p16580_C/T,p16580_T/T,mdm2_G/G,mdm2_G/T,mdm2_T/T,GAL3_A/A,GAL3_C/A,GAL3_C/C,TIM1_C/C,TIM1_G/C,TIM1_G/G
0,1,49,False,True,False,True,False,False,True,False,False,False,True,False,False,True,False
1,1,49,False,True,False,True,False,False,True,False,False,True,False,False,False,True,False
2,1,49,False,True,False,True,False,False,False,True,False,False,True,False,True,False,False
3,1,36,False,True,False,True,False,False,False,False,True,False,True,False,True,False,False
4,1,49,True,False,False,False,True,False,False,False,True,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,0,81,False,True,False,True,False,False,False,False,True,False,False,True,False,False,True
111,0,73,True,False,False,True,False,False,False,False,True,False,False,True,False,False,True
112,0,56,True,False,False,True,False,False,False,True,False,False,False,True,False,True,False
113,0,74,False,True,False,False,True,False,True,False,False,False,False,True,False,False,True


In [3]:
X=df.drop('Class',axis=1)
y=df['Class']

In [4]:

#split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
random_state=123

In [5]:
#overampling
sm = SMOTE(random_state=random_state)
X_oversampled, y_oversampled = sm.fit_resample(X, y)

X_train_oversampled, X_test_oversampled, y_train_oversampled, y_test_oversampled = train_test_split(X_oversampled, y_oversampled, test_size=0.2, random_state=42)


In [6]:
#underampling
rus = RandomUnderSampler(random_state=random_state)
X_undersampled, y_undersampled = rus.fit_resample(X, y)
X_train_undersampled, X_test_undersampled, y_train_undersampled, y_test_undersampled = train_test_split(X_undersampled, y_undersampled, test_size=0.2, random_state=42)

In [9]:

def model_training(X_train, X_test, y_train, y_test, random_state=42):
    # List of models to train
    models = [
        AdaBoostClassifier(),
        ExtraTreesClassifier(random_state=random_state, n_jobs=-1), # Parallel processing enabled
        HistGradientBoostingClassifier(random_state=random_state),
        LGBMClassifier(),
        LogisticRegression(random_state=random_state, n_jobs=-1), # Parallel processing enabled
        RandomForestClassifier(max_depth=10, random_state=random_state, n_estimators=100, n_jobs=-1), # Parallel processing enabled
        SVC(random_state=random_state, kernel='linear', C=1),
        XGBClassifier(random_state=random_state)
    ]

    # Create an empty DataFrame to store results
    results_df = pd.DataFrame(columns=['Model', 'Accuracy'])

    for model in models:
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        predictions = model.predict(X_test)
        
        # Evaluate the model
        accuracy = accuracy_score(y_test, predictions)
        
        # Append results to the DataFrame
        temp_df = pd.DataFrame({'Model': [model.__class__.__name__], 'Accuracy': [accuracy]})
        results_df = pd.concat([results_df, temp_df], ignore_index=True)


    # Sort and return the results DataFrame
    return results_df.sort_values(by=['Accuracy'], ascending=False)


In [10]:
model_training(X_train, X_test, y_train, y_test)

/var/folders/k4/_ms9gszn46s_6f86x4tywvbc0000gn/T/ipykernel_81744/3984518863.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, temp_df], ignore_index=True)


[LightGBM] [Info] Number of positive: 40, number of negative: 52
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000299 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 38
[LightGBM] [Info] Number of data points in the train set: 92, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.434783 -> initscore=-0.262364
[LightGBM] [Info] Start training from score -0.262364
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Lig

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Model,Accuracy
0,AdaBoostClassifier,0.869565
4,LogisticRegression,0.826087
6,SVC,0.826087
1,ExtraTreesClassifier,0.782609
2,HistGradientBoostingClassifier,0.782609
3,LGBMClassifier,0.782609
7,XGBClassifier,0.782609
5,RandomForestClassifier,0.739130


In [11]:
model_training(X_train_oversampled, X_test_oversampled, y_train_oversampled, y_test_oversampled)

/var/folders/k4/_ms9gszn46s_6f86x4tywvbc0000gn/T/ipykernel_81744/3984518863.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, temp_df], ignore_index=True)


[LightGBM] [Info] Number of positive: 51, number of negative: 53
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000558 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 43
[LightGBM] [Info] Number of data points in the train set: 104, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.490385 -> initscore=-0.038466
[LightGBM] [Info] Start training from score -0.038466
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Model,Accuracy
2,HistGradientBoostingClassifier,0.884615
3,LGBMClassifier,0.884615
0,AdaBoostClassifier,0.846154
1,ExtraTreesClassifier,0.846154
4,LogisticRegression,0.846154
6,SVC,0.846154
5,RandomForestClassifier,0.807692
7,XGBClassifier,0.730769


In [12]:
model_training(X_train_undersampled, X_test_undersampled, y_train_undersampled, y_test_undersampled)

/var/folders/k4/_ms9gszn46s_6f86x4tywvbc0000gn/T/ipykernel_81744/3984518863.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, temp_df], ignore_index=True)


[LightGBM] [Info] Number of positive: 42, number of negative: 38
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000487 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 35
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.525000 -> initscore=0.100083
[LightGBM] [Info] Start training from score 0.100083
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

,Model,Accuracy
3,LGBMClassifier,0.90
6,SVC,0.90
1,ExtraTreesClassifier,0.85
2,HistGradientBoostingClassifier,0.85
4,LogisticRegression,0.85
5,RandomForestClassifier,0.85
7,XGBClassifier,0.85
0,AdaBoostClassifier,0.75


In [13]:
model_params = {
    
    'AdaBoostClassifier' : {
        'model': AdaBoostClassifier(),
        'params': {
            'n_estimators': [50, 100, 200], 
            'learning_rate': [0.01, 0.1, 1]
        }
    } ,
    
    'ExtraTreesClassifier' : {
        'model': ExtraTreesClassifier(),
        'params': {
        'n_estimators': [50, 100, 200],
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 4, 6],
        'min_samples_leaf': [1, 2, 3],
        'max_features': ['auto', 'sqrt', 'log2']
        }
    },
    'LGBMClassifier' : {
        'model': LGBMClassifier(),
        'params': {
            'boosting_type': ['gbdt', 'dart', 'goss', 'rf'],
            'learning_rate': [0.01, 0.05, 0.1, 0.5, 1],
            'n_estimators': [50, 100, 200],
            'num_leaves': [15, 31, 63, 127],
            'min_data_in_leaf': [5, 10, 20],
            'lambda_l2': [0.0, 0.1, 1.0]
        }
    },
    'SVM':{
        'model': SVC(),
        'params': {
            'C': [0.1, 1, 10, 100], 
            'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
        }
    },
    'XGBClassifier' : {
        'model': XGBClassifier(),
        'params': {
            'n_estimators': [50, 100, 200], 
            'learning_rate': [0.01, 0.1, 1],
            'max_depth': [3, 5, 10],
            'gamma': [0, 0.1, 0.5, 1],
            'subsample': [0.5, 0.75, 1],
            'colsample_bytree': [0.5, 0.75, 1],
            'lambda': [0.0, 0.1, 1.0]
        }
    },
    'RandomForestClassifier' : {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 4, 6],
            'min_samples_leaf': [1, 2, 3],
            'max_features': ['auto', 'sqrt', 'log2']
        }
    },
    'HistGradientBoostingClassifier':{
        'model': HistGradientBoostingClassifier(),
        'params': {
            'learning_rate': [0.01, 0.05, 0.1, 0.5, 1],
            'max_iter': [50, 100, 200],
            'max_leaf_nodes': [15, 31, 63, 127],
            'min_samples_leaf': [5, 10, 20],
            'l2_regularization': [0.0, 0.1, 1.0]
        }
    },
    'LogisticRegression' : {
        'model': LogisticRegression(),
        'params': {
            'C': [0.1, 1, 10, 100], 
            'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
        }
    }
    
}

In [ ]:
scores = []
n_splits = 15
cv=KFold(n_splits=n_splits, random_state=random_state, shuffle=True)
for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=cv, return_train_score=False,n_jobs=-1,error_score=0)
    clf.fit(X_train,y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
